In [1]:
import sys
import os

sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
sys.path.insert(0, r"E:/Thesis/crypto_project")

In [2]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "database_server.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django

django.setup()

In [3]:
from crypto_calculator.models import RawPriceData
from  crypto_calculator.serializers import RawPriceDataSerializer

In [4]:
help(RawPriceData)

Help on class RawPriceData in module crypto_calculator.models:

class RawPriceData(django.db.models.base.Model)
 |  RawPriceData(*args, **kwargs)
 |  
 |  RawPriceData(id, open, high, low, close, volume, dividends, stock_splits, date, symbol)
 |  
 |  Method resolution order:
 |      RawPriceData
 |      django.db.models.base.Model
 |      django.db.models.utils.AltersData
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  close = <django.db.models.query_utils.DeferredAttribute object>
 |  date = <django.db.models.query_utils.DeferredAttribute object>
 |  dividends = <django.db.models.query_utils.DeferredAttribute object>
 |  high = <django.db.models.query_utils.DeferredAttribute object>
 |  id = <django.db.models.query_utils.DeferredAttribute object>
 |  low = <django.db.models.query_utils.DeferredAttribute object>
 |  open = <django.db.models.query_utils.DeferredAttribute object>
 |  stock_splits = <django.db.models.query_

In [5]:
raw_price_data = RawPriceData.objects.filter(symbol='1000SATS-USD')
print(help(raw_price_data))

Help on QuerySet in module django.db.models.query object:

class QuerySet(django.db.models.utils.AltersData)
 |  QuerySet(model=None, query=None, using=None, hints=None)
 |  
 |  Represent a lazy database lookup for a set of objects.
 |  
 |  Method resolution order:
 |      QuerySet
 |      django.db.models.utils.AltersData
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __aiter__(self)
 |  
 |  __and__(self, other)
 |  
 |  __bool__(self)
 |  
 |  __deepcopy__(self, memo)
 |      Don't populate the QuerySet's cache.
 |  
 |  __getitem__(self, k)
 |      Retrieve an item or slice from the set of results.
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, model=None, query=None, using=None, hints=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      The queryset iterator protocol uses three nested iterators in the
 |      default case:
 |          1. sql.compiler.execute_sql()
 |             - Returns 100 rows at t

In [6]:
for k in raw_price_data:
    print(f"{k.id}/{k.open}/{k.close}/{k.symbol}/{k.date}")

0/0.0006200000061653554/0.0005520000122487545/1000SATS-USD/2023-12-12
1/0.0005530000198632479/0.0005119999987073243/1000SATS-USD/2023-12-13
2/0.0005119999987073243/0.0007580000092275441/1000SATS-USD/2023-12-14
3/0.0007570000016130507/0.0007329999934881926/1000SATS-USD/2023-12-15
4/0.0007340000011026859/0.0007769999792799354/1000SATS-USD/2023-12-16
5/0.0007769999792799354/0.0007559999939985573/1000SATS-USD/2023-12-17
6/0.0007559999939985573/0.0006849999772384763/1000SATS-USD/2023-12-18
7/0.0006840000278316438/0.0006980000180192292/1000SATS-USD/2023-12-19
8/0.0006980000180192292/0.0007070000283420086/1000SATS-USD/2023-12-20
9/0.0007070000283420086/0.0007110000005923212/1000SATS-USD/2023-12-21
10/0.0007110000005923212/0.0006520000169984996/1000SATS-USD/2023-12-22
11/0.0006520000169984996/0.0006419999990612268/1000SATS-USD/2023-12-23
12/0.0006419999990612268/0.0006300000241026282/1000SATS-USD/2023-12-24
13/0.0006300000241026282/0.0008779999916441739/1000SATS-USD/2023-12-25
14/0.00087699998

In [7]:
# serialized_price_data=RawPriceDataSerializer(raw_price_data)
# serialized_price_data.data

In [8]:
help(raw_price_data)

Help on QuerySet in module django.db.models.query object:

class QuerySet(django.db.models.utils.AltersData)
 |  QuerySet(model=None, query=None, using=None, hints=None)
 |  
 |  Represent a lazy database lookup for a set of objects.
 |  
 |  Method resolution order:
 |      QuerySet
 |      django.db.models.utils.AltersData
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __aiter__(self)
 |  
 |  __and__(self, other)
 |  
 |  __bool__(self)
 |  
 |  __deepcopy__(self, memo)
 |      Don't populate the QuerySet's cache.
 |  
 |  __getitem__(self, k)
 |      Retrieve an item or slice from the set of results.
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, model=None, query=None, using=None, hints=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      The queryset iterator protocol uses three nested iterators in the
 |      default case:
 |          1. sql.compiler.execute_sql()
 |             - Returns 100 rows at t